
# 🗣🎙️CosyVoice Zero-Shot Voice Cloning 🎙️
### Credit
[CosyVoice Github](https://github.com/FunAudioLLM/CosyVoice) <br>
[Hugging Face Space](https://huggingface.co/spaces/modelscope/CosyVoice-300M) <br>
[CosyVoice-300M Model Download](https://www.modelscope.cn/models/iic/CosyVoice-300M)
<br>
## Disclaimer
The content provided below is for academic purposes only and is intended to demonstrate technical capabilities.


In [ ]:
#@title Automatic Install CosyVoice, Download Model & AUTO-RESTART (Cancel ```Restart session``` Pop UP)
import os
import shutil
!pip install modelscope

# root_path="." #for windows/kaggle/mac
# root_path=os.getcwd() #else use this one
root_path="/content" #if you are not running this on google colab comment this

base_path=f"{root_path}"
os.chdir(base_path)
if os.path.exists(f"{base_path}/CosyVoice"):
  shutil.rmtree(f"{base_path}/CosyVoice")
  print(f"Deleting Old {base_path}/CosyVoice")
!git clone --recursive https://github.com/FunAudioLLM/CosyVoice.git
os.chdir(f"{base_path}/CosyVoice")

#Downloading Model using git clone is very slow
# !git clone https://www.modelscope.cn/iic/CosyVoice-300M.git pretrained_models/CosyVoice-300M
from modelscope import snapshot_download
snapshot_download('iic/CosyVoice-300M', local_dir=f'{base_path}/CosyVoice/pretrained_models/CosyVoice-300M')

!pip install -r requirements.txt
!pip install matcha-tts
!echo /usr/lib64-nvidia/ >/etc/ld.so.conf.d/libcuda.conf; ldconfig
!pip install pydub==0.25.1
!pip install pysrt==1.1.2
!pip install librosa==0.10.2.post1
from IPython.display import clear_output
clear_output()
import time
time.sleep(5)
import os
os.kill(os.getpid(), 9)

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.7/40.7 kB 1.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.8/5.8 MB 32.7 MB/s eta 0:00:00
Cloning into 'CosyVoice'...
remote: Enumerating objects: 1192, done.
remote: Counting objects: 100% (634/634), done.
remote: Compressing objects: 100% (226/226), done.
remote: Total 1192 (delta 534), reused 408 (delta 408), pack-reused 558 (from 1)
Receiving objects: 100% (1192/1192), 1.31 MiB | 4.33 MiB/s, done.
Resolving deltas: 100% (732/732), done.
Submodule 'third_party/Matcha-TTS' (https://github.com/shivammehta25/Matcha-TTS.git) registered for path 'third_party/Matcha-TTS'
Cloning into '/content/CosyVoice/third_party/Matcha-TTS'...
remote: Enumerating objects: 921, done.        
remote: Counting objects: 100% (406/406), done.        
remote: Compressing objects: 100% (124/124), done.        
remote: Total 921 (delta 336), reused 283 (delta 282), pack-reused 515 (from 1)        
Receiving objects: 100% (921/921), 64.08 MiB | 

Don't panic if Google Colab got disconnected run from the next cell

In [ ]:
#@title <-- Tap this if you play on Mobile { display-mode: "form" }

%%html
<b>Press play on the music player to keep the tab alive, then run the cell below</b><br/>
<audio src="https://raw.githubusercontent.com/KoboldAI/KoboldAI-Client/main/colab/silence.m4a" controls>

In [ ]:
#@title import CosyVoice model
%cd /content/CosyVoice
from cosyvoice.cli.cosyvoice import CosyVoice
from cosyvoice.utils.file_utils import load_wav
import torchaudio

cosyvoice = CosyVoice('/content/CosyVoice/pretrained_models/CosyVoice-300M')
from IPython.display import clear_output
clear_output()
#@title utils
import librosa
import whisper
from pydub import AudioSegment
import os
import random
import numpy as np
import uuid
import torch
import shutil
from google.colab import files

import nltk
nltk.download('punkt')
from nltk.tokenize import sent_tokenize
from IPython.display import Audio
from IPython.core.display import display
from tqdm.notebook import tqdm

select_model ="tiny" # ['tiny', 'base']
whisper_model = whisper.load_model(select_model)
language_dict = {
    "Chinese": "zh",
    "English": "en",
    "Japanese": "ja",
    "Cantonese": "yue",
    "Korean": "ko"
}
def convert_to_wav(file_path):
    # Check if the file is already a WAV file
    if file_path.lower().endswith('.wav'):
        # print(f"{file_path} is already a WAV file.")
        return file_path

    # Define the output file path with .wav extension
    output_file_path = os.path.splitext(file_path)[0] + '.wav'

    # Convert the file to WAV format
    audio = AudioSegment.from_file(file_path)
    audio.export(output_file_path, format='wav')
    # print(f"Converted {file_path} to {output_file_path}.")

    return output_file_path

def set_all_random_seed(seed):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)

def postprocess(wav_file_path, Language,audio_to_text=False,top_db=60, hop_length=220, win_length=440):
    global language_dict
    max_val = 0.8
    prompt_sr, target_sr = 16000, 22050
    default_data = np.zeros(target_sr)
    if torchaudio.info(wav_file_path).sample_rate < prompt_sr:
      prompt_sr=target_sr
    speech=load_wav(wav_file_path, prompt_sr)
    speech, _ = librosa.effects.trim(
        speech, top_db=top_db,
        frame_length=win_length,
        hop_length=hop_length
    )
    if speech.abs().max() > max_val:
        speech = speech / speech.abs().max() * max_val
    speech = torch.concat([speech, torch.zeros(1, int(target_sr * 0.2))], dim=1)
    #extract text
    if audio_to_text:
      result = whisper_model.transcribe(wav_file_path,language=language_dict[Language])
      prompt_text=result["text"].strip()
    else:
      prompt_text=""
    # print(f"Reference Audio Text:\n{prompt_text}")
    return speech,prompt_text
from pydub import AudioSegment

def merge_audio(audio_chunks_list, save_path):
    # Initialize an empty AudioSegment object for the final output
    merged_audio = AudioSegment.empty()

    # Loop through the list of audio file paths
    for audio_path in audio_chunks_list:
        # Load each audio file
        audio = AudioSegment.from_file(audio_path)
        # Append the audio to the merged_audio object
        merged_audio += audio

    # Export the merged audio to the specified path in WAV format
    merged_audio.export(save_path, format="wav")


def chunks_sentences(paragraph, join_limit=2):
    sentences = sent_tokenize(paragraph)
    # Initialize an empty list to store the new sentences
    new_sentences = []

    # Iterate through the list of sentences in steps of 'join_limit'
    for i in range(0, len(sentences), join_limit):
        # Join the sentences with a space between them
        new_sentence = ' '.join(sentences[i:i + join_limit])
        new_sentences.append(new_sentence)

    return new_sentences

def speed_change(clone_voice_save_path,speedup_factor):
  speedup_filename=clone_voice_save_path.replace(".wav","_speed_up.wav")
  speed_change_command=f"ffmpeg -i {clone_voice_save_path} -filter:a atempo={speedup_factor} {speedup_filename} -y"
  var=os.system(speed_change_command)
  if var==0:
    return speedup_filename
  else:
    print(speed_change_command)
    return None


def voice_clone(text,reference_audio_file,Language="English",clone_method="3s Quick Clone",seed=0,save_folder="."):
  global language_dict
  if save_folder.endswith("/"):
    save_folder=save_folder[:-1]
  if not os.path.exists(save_folder):
    os.mkdir(save_folder)
  small_sentences=chunks_sentences(text)
  #may be I am wrong
  seed=seed+len(small_sentences)
  set_all_random_seed(seed)
  wav_file_path=convert_to_wav(reference_audio_file)


  if 'COLAB_GPU' in os.environ:
    tts_save_folder="/content/cosy_tts"
  else:
    tts_save_folder="./cosy_tts"
  if os.path.exists(tts_save_folder):
    shutil.rmtree(tts_save_folder)
  os.mkdir(tts_save_folder)
  if clone_method=="3s Quick Clone":
    prompt_speech_16k,prompt_text = postprocess(wav_file_path,Language,audio_to_text=True)
    print(f"Reference Audio Text:\n{prompt_text}")
  else:
    prompt_speech_16k,prompt_text = postprocess(wav_file_path,Language,audio_to_text=False)
  audio_chunks_list=[]
  for index, tts_text in tqdm(enumerate(small_sentences), total=len(small_sentences)):
  # for index, tts_text in enumerate(small_sentences):
    if clone_method=="3s Quick Clone":
      output = cosyvoice.inference_zero_shot(tts_text, prompt_text, prompt_speech_16k)
    else:
      tts_text=f"<|{language_dict['English']}|> {tts_text}"
      output = cosyvoice.inference_cross_lingual(tts_text, prompt_speech_16k)
    temp_file_path=f"{tts_save_folder}/{index}.wav"
    torchaudio.save(temp_file_path, output['tts_speech'], 22050)
    audio_chunks_list.append(temp_file_path)
  file_name=text[:20].replace(" ","_")
  file_name=file_name.replace("<|","").replace("|>","")
  random_uuid = str(uuid.uuid4())[:6]
  save_clone_voice_path=f"{save_folder}/{file_name}_{random_uuid}.wav"
  if len(audio_chunks_list)==0:
    return None
  elif len(audio_chunks_list)==1:
    shutil.copy(audio_chunks_list[-1],save_clone_voice_path)
    return save_clone_voice_path
  else:
    merge_audio(audio_chunks_list, save_clone_voice_path)
    return save_clone_voice_path

from IPython.display import clear_output
clear_output()

[Online Audio Cutter](https://mp3cut.net/) &emsp;If you have a larger Audio<br>
[Vocal & Music Seperator](https://vocalremover.org/)&emsp;If you have a audio with background Music<br>
[Enhance low quality audio](https://podcast.adobe.com/)&emsp;If your audio file have poor audio quality

In [ ]:
#@title Upload 6s Audio clip
import os
from google.colab import files
%cd /content/
uploaded = files.upload()
%cd /content/CosyVoice/
upload_list=[]
for fn in uploaded.keys():
  upload_list.append('/content/'+fn)
from IPython.display import clear_output
clear_output()
upload_list[-1]

'/content/my_voice.mp3'

In [ ]:
#@title Run CosyVoice for voice cloning (Support Long Text)
text="Sometimes the stories that don't matter are the ones that matter the most."# @param {type: "string"}
Reference_Audio_File="/content/my_voice.mp3"# @param {type: "string"}
Language = "English" # @param [ 'English','Chinese', 'Japanese', 'Cantonese', 'Korean'] {allow-input: true}
Clone_Method = "3s Quick Clone" # @param ["3s Quick Clone", "Cross-lingual Clone"] {allow-input: true}

Seed=100 # @param {type: "number"}
# save_folder="/content/clone_voice" # @param {type: "string"}
save_folder="/content/clone_voice"
clone_voice_save_path=voice_clone(text,Reference_Audio_File,Language,Clone_Method,Seed,save_folder)


In [ ]:
#@title Display Original & Cloned Voice
from pydub import AudioSegment

def audio_duration_check(file_path):
    audio = AudioSegment.from_file(file_path)
    duration_ms = len(audio)  # Duration in milliseconds
    duration_s = duration_ms / 1000  # Convert to seconds
    return duration_s <= 240  # 4 minutes in seconds

print(f"Original Audio File:")
if audio_duration_check(Reference_Audio_File):
  display(Audio(Reference_Audio_File, autoplay=False))
else:
  print("Audio File is larger please download the audio in your local device")
  print(f"Save at {Reference_Audio_File}")

print(f"Cloned Audio File:")
if audio_duration_check(clone_voice_save_path):
  display(Audio(clone_voice_save_path, autoplay=False))
else:
  print("Audio File is larger please download the audio in your local device")
  print(f"Save at {clone_voice_save_path}")
  files.download(clone_voice_save_path)


In [ ]:
#@title Download Cloned Voice
from google.colab import files
print(f"Save at {clone_voice_save_path}")
files.download(clone_voice_save_path)

## Change Audio Speed

In [ ]:
#@title Change the audio speed to achieve your desired voice
speed="1.2"  # @param {type: "string"}
speedup_factor=float(speed)
speed_change_file=speed_change(clone_voice_save_path,speedup_factor)

print(f"Cloned Audio File:")
if audio_duration_check(clone_voice_save_path):
  display(Audio(clone_voice_save_path, autoplay=False))
else:
  print("Audio File is larger please download the audio in your local device")
  print(f"Save at {clone_voice_save_path}")
  files.download(clone_voice_save_path)

print(f"Cloned Audio File After Speed Change:")
if audio_duration_check(speed_change_file):
  display(Audio(speed_change_file, autoplay=False))
else:
  print("Audio File is larger please download the audio in your local device")
  print(f"Save at {speed_change_file}")
  files.download(speed_change_file)

In [ ]:
#@title Download the speed-changed cloned voice
from google.colab import files
print(f"Save at {clone_voice_save_path}")
files.download(clone_voice_save_path)

In [ ]:
#@title Use as function

# voice_clone(text="hi how are you ",reference_audio_file='/content/audio.wav',Language='English',clone_method="3s Quick Clone",seed=0,save_folder="/content/clone_voice")

In [ ]:
#@title CosyVoice Funtion for subtitle dubbing
# temp_path=voice_clone_for_subtitle(text,prompt_speech_16k,prompt_text,Language,"3s Quick Clone",Seed,"/content/clone_voice")
def voice_clone_for_subtitle(text,prompt_speech_16k,prompt_text,Language="English",clone_method="3s Quick Clone",seed=0,save_folder="."):
  global language_dict
  if save_folder.endswith("/"):
    save_folder=save_folder[:-1]
  if not os.path.exists(save_folder):
    os.mkdir(save_folder)
  small_sentences=chunks_sentences(text)
  #may be I am wrong
  seed=seed+len(small_sentences)
  set_all_random_seed(seed)
  if 'COLAB_GPU' in os.environ:
    tts_save_folder="/content/cosy_tts"
  else:
    tts_save_folder="./cosy_tts"
  if os.path.exists(tts_save_folder):
    shutil.rmtree(tts_save_folder)
  os.mkdir(tts_save_folder)
  audio_chunks_list=[]
  # for index, tts_text in tqdm(enumerate(small_sentences), total=len(small_sentences)):
  for index, tts_text in enumerate(small_sentences):
    if clone_method=="3s Quick Clone":
      output = cosyvoice.inference_zero_shot(tts_text, prompt_text, prompt_speech_16k)
    else:
      tts_text=f"<|{language_dict['English']}|> {tts_text}"
      output = cosyvoice.inference_cross_lingual(tts_text, prompt_speech_16k)
    temp_file_path=f"{tts_save_folder}/{index}.wav"
    torchaudio.save(temp_file_path, output['tts_speech'], 22050)
    audio_chunks_list.append(temp_file_path)
  file_name=text[:20].replace(" ","_")
  file_name=file_name.replace("<|","").replace("|>","")
  bad_list=["/",".","'",'"']
  for i in bad_list:
    file_name=file_name.replace(i,"")
  random_uuid = str(uuid.uuid4())[:6]
  save_clone_voice_path=f"{save_folder}/{file_name}_{random_uuid}.wav"
  if len(audio_chunks_list)==0:
    return None
  elif len(audio_chunks_list)==1:
    shutil.copy(audio_chunks_list[-1],save_clone_voice_path)
    return save_clone_voice_path
  else:
    merge_audio(audio_chunks_list, save_clone_voice_path)
    return save_clone_voice_path

def your_tts(text,audio_path,language):
  global Reference_Audio_File,Language,Clone_Method,Seed
  wav_file_path=convert_to_wav(Reference_Audio_File)
  if Clone_Method=="3s Quick Clone":
    prompt_speech_16k,prompt_text = postprocess(wav_file_path,Language,audio_to_text=True)
  else:
    prompt_speech_16k,prompt_text = postprocess(wav_file_path,Language,audio_to_text=False)
  temp_path=voice_clone_for_subtitle(text,prompt_speech_16k,prompt_text,Language,"3s Quick Clone",Seed,"/content/clone_voice")
  shutil.copy(temp_path,audio_path)

### Subtitle Dubbing (Need update, for now it's very slow)

In [ ]:
#@title Mount Google Drive to autosave the generated audio file
from google.colab import drive
import os
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
#@title Upload a Subtitle
import os
from google.colab import files
%cd /content/
uploaded = files.upload()
%cd /content/CosyVoice/
upload_list=[]
for fn in uploaded.keys():
  upload_list.append('/content/'+fn)
from IPython.display import clear_output
clear_output()
upload_list[-1]

'/content/sub.srt'

In [ ]:
#@title Generate Audio File From Subtitle

from tqdm.notebook import tqdm

import os
def get_subtitle_Dub_path(srt_file_path,Language):
  file_name = os.path.splitext(os.path.basename(srt_file_path))[0]
  if not os.path.exists("/content/TTS_DUB"):
    os.mkdir("/content/TTS_DUB")
  new_path=f"/content/TTS_DUB/{Language}_{file_name}.wav"
  return new_path






import subprocess
import json

def get_video_duration(video_path):
    try:
        # Run ffmpeg command to get video information in JSON format
        result = subprocess.run(
            ['ffmpeg', '-i', video_path, '-f', 'ffmetadata', '-'],
            stderr=subprocess.PIPE,
            text=True
        )

        # Parse the duration from the stderr output
        for line in result.stderr.split('\n'):
            if 'Duration' in line:
                duration_str = line.split('Duration: ')[1].split(',')[0]
                h, m, s = duration_str.split(':')
                duration = int(h) * 3600 + int(m) * 60 + float(s)
                return duration
    except Exception as e:
        print(f"Error: {e}")
        return None


def replace_audio(video_path,audio_path):
  if not video_path.lower().endswith(".mp4"):
    return
  tts_audio = AudioSegment.from_file(dub_save_path)
  audio_duration = len(tts_audio)/1000
  video_duration=get_video_duration(video_path)
  slience_duration=video_duration-audio_duration
  audio_segment = AudioSegment.from_file(audio_path)
  slience_Segment= AudioSegment.silent(duration=slience_duration)
  marge_audio=audio_segment+slience_Segment
  marge_audio.export("/content/new_audio.wav", format="wav")
  command=f"ffmpeg -i {video_path}  -i /content/new_audio.wav -map 0:v -map 1:a -c:v copy -shortest /content/output.mp4 -y"
  var=os.system(command)
  if var==0:
    if os.path.exists("/content/gdrive/MyDrive/upload"):
      file_name = os.path.basename(video_path)
      shutil.copy("/content/output.mp4", f"/content/gdrive/MyDrive/upload/change_audio_{file_name}")
      print(f"Copied at drive '/content/gdrive/MyDrive/upload/change_audio_{file_name}'")
      return f"/content/gdrive/MyDrive/upload/change_audio_{file_name}"
  else:
    print(command)
    return None





import pysrt

def clean_srt(input_path):
    file_name = os.path.basename(input_path)
    output_folder = "/content/save_srt"
    if not os.path.exists(output_folder):
        os.mkdir(output_folder)
    output_path = f"{output_folder}/{file_name}"

    def clean_srt_line(text):
        bad_list = ["[", "]", "♫", "\n"]
        for i in bad_list:
            text = text.replace(i, "")
        return text.strip()

    # Load the subtitle file
    subs = pysrt.open(input_path)

    # Iterate through each subtitle and print its details
    with open(output_path, "w", encoding='utf-8') as file:
        for sub in subs:
            file.write(f"{sub.index}\n")
            file.write(f"{sub.start} --> {sub.end}\n")
            file.write(f"{clean_srt_line(sub.text)}\n")
            file.write("\n")
        file.close()
    # print(f"Clean SRT saved at: {output_path}")
    return output_path
# Example usage



###

from pydub import AudioSegment
import shutil
import subprocess
import os
import uuid
import re




# dub_save_path=get_subtitle_Dub_path(srt_file_path,Language)
import time

# def your_tts(text,audio_path,language):
#   global Reference_Audio_File,Language,Clone_Method,Seed
#   wav_file_path=convert_to_wav(Reference_Audio_File)
#   if Clone_Method=="3s Quick Clone":
#     prompt_speech_16k,prompt_text = postprocess(wav_file_path,Language,audio_to_text=True)
#   else:
#     prompt_speech_16k,prompt_text = postprocess(wav_file_path,Language,audio_to_text=False)
#   temp_path=voice_clone_for_subtitle(text,prompt_speech_16k,prompt_text,Language,"3s Quick Clone",Seed,"/content/clone_voice")
#   shutil.copy(temp_path,audio_path)

class SRTDubbing:
    def __init__(self):
        pass

    @staticmethod
    def text_to_speech(text, audio_path, language, actual_duration):
        tts_filename = "temp.wav"
        your_tts(text,tts_filename,language)
        # Check the duration of the generated TTS audio
        tts_audio = AudioSegment.from_file(tts_filename)
        tts_duration = len(tts_audio)

        if actual_duration == 0:
            # If actual duration is zero, use the original TTS audio without modifications
            shutil.move(tts_filename, audio_path)
            return

        # If TTS audio duration is longer than actual duration, speed up the audio
        if tts_duration > actual_duration:
            speedup_factor = tts_duration / actual_duration
            speedup_filename = "speedup_temp.wav"

            # Use ffmpeg to change audio speed
            subprocess.run([
                "ffmpeg",
                "-i", tts_filename,
                "-filter:a", f"atempo={speedup_factor}",
                speedup_filename
            ], check=True)

            # Replace the original TTS audio with the sped-up version
            shutil.move(speedup_filename, audio_path)
        elif tts_duration < actual_duration:
            # If TTS audio duration is less than actual duration, add silence to match the duration
            silence_gap = actual_duration - tts_duration
            silence = AudioSegment.silent(duration=int(silence_gap))
            new_audio = tts_audio + silence

            # Save the new audio with added silence
            new_audio.export(audio_path, format="wav")
        else:
            # If TTS audio duration is equal to actual duration, use the original TTS audio
            shutil.move(tts_filename, audio_path)

    @staticmethod
    def make_silence(pause_time, pause_save_path):
        silence = AudioSegment.silent(duration=pause_time)
        silence.export(pause_save_path, format="wav")
        return pause_save_path

    @staticmethod
    def create_folder_for_srt(srt_file_path):
        srt_base_name = os.path.splitext(os.path.basename(srt_file_path))[0]
        random_uuid = str(uuid.uuid4())[:4]
        dummy_folder_path = "/content/dummy"
        if not os.path.exists(dummy_folder_path):
            os.makedirs(dummy_folder_path)
        folder_path = os.path.join(dummy_folder_path, f"{srt_base_name}_{random_uuid}")
        os.makedirs(folder_path, exist_ok=True)
        return folder_path

    @staticmethod
    def concatenate_audio_files(audio_paths, output_path):
        concatenated_audio = AudioSegment.silent(duration=0)
        for audio_path in audio_paths:
            audio_segment = AudioSegment.from_file(audio_path)
            concatenated_audio += audio_segment
        concatenated_audio.export(output_path, format="wav")

    def srt_to_dub(self, srt_file_path, dub_save_path,language='en'):
        result = self.read_srt_file(srt_file_path)
        new_folder_path = self.create_folder_for_srt(srt_file_path)
        join_path = []
        for i in tqdm(result):
        # for i in result:
            text = i['text']
            actual_duration = i['end_time'] - i['start_time']
            pause_time = i['pause_time']
            slient_path = f"{new_folder_path}/{i['previous_pause']}"
            self.make_silence(pause_time, slient_path)
            join_path.append(slient_path)
            tts_path = f"{new_folder_path}/{i['audio_name']}"
            self.text_to_speech(text, tts_path, language, actual_duration)
            join_path.append(tts_path)
        self.concatenate_audio_files(join_path, dub_save_path)

    @staticmethod
    def convert_to_millisecond(time_str):
      if isinstance(time_str, str):
          hours, minutes, second_millisecond = time_str.split(':')
          seconds, milliseconds = second_millisecond.split(",")

          total_milliseconds = (
              int(hours) * 3600000 +
              int(minutes) * 60000 +
              int(seconds) * 1000 +
              int(milliseconds)
          )

          return total_milliseconds
    @staticmethod
    def read_srt_file(file_path):
        entries = []
        default_start = 0
        previous_end_time = default_start
        entry_number = 1
        audio_name_template = "{}.wav"
        previous_pause_template = "{}_before_pause.wav"

        with open(file_path, 'r', encoding='utf-8') as file:
            lines = file.readlines()
            # print(lines)
            for i in range(0, len(lines), 4):
                time_info = re.findall(r'(\d+:\d+:\d+,\d+) --> (\d+:\d+:\d+,\d+)', lines[i + 1])
                start_time = SRTDubbing.convert_to_millisecond(time_info[0][0])
                end_time = SRTDubbing.convert_to_millisecond(time_info[0][1])

                current_entry = {
                    'entry_number': entry_number,
                    'start_time': start_time,
                    'end_time': end_time,
                    'text': lines[i + 2].strip(),
                    'pause_time': start_time - previous_end_time if entry_number != 1 else start_time - default_start,
                    'audio_name': audio_name_template.format(entry_number),
                    'previous_pause': previous_pause_template.format(entry_number),
                }

                entries.append(current_entry)
                previous_end_time = end_time
                entry_number += 1

        return entries

# Example usage
Enter_srt_file_path = '/content/sub.srt'  # @param {type: "string"}
srt_file_path=clean_srt(Enter_srt_file_path)

Reference_Audio_File="/content/my_voice.mp3"# @param {type: "string"}
Language = "English" # @param [ 'English','Chinese', 'Japanese', 'Cantonese', 'Korean'] {allow-input: true}
Clone_Method = "3s Quick Clone" # @param ["3s Quick Clone", "Cross-lingual Clone"] {allow-input: true}
Seed=100000 # @param {type: "number"}


srt_dubbing = SRTDubbing()
current_language=Language
dub_save_path=get_subtitle_Dub_path(srt_file_path,Language)
srt_dubbing.srt_to_dub(srt_file_path, dub_save_path,current_language)
from IPython.display import clear_output
clear_output()
print(f"{Language} Dub Audio File Save At : {dub_save_path}")
if os.path.exists("/content/gdrive/MyDrive/upload"):
  file_name = os.path.basename(dub_save_path)
  shutil.copy(dub_save_path, f"/content/gdrive/MyDrive/upload/{file_name}")
  print(f"Copied at drive '/content/gdrive/MyDrive/upload/{file_name}'")


# replace_audio_path=replace_audio(audio_file_path,dub_save_path)

from google.colab import files

files.download(dub_save_path)
# files.download(replace_audio_path)



  0%|          | 0/157 [00:00<?, ?it/s]